In [1]:
from skt.vault_utils import get_secrets
proxies = get_secrets('proxies')

In [2]:
from skt.gcp import (
    PROJECT_ID,
    bq_insert_overwrite,
    bq_to_df,
    bq_to_pandas,
    df_to_bq_table,
    get_bigquery_client,
    bq_table_exists,
    get_max_part,
    load_query_result_to_table,
    pandas_to_bq,
    pandas_to_bq_table,
    load_bigquery_ipython_magic,
    get_bigquery_client,
    _print_query_job_results,
    load_query_result_to_partitions
    
)

from skt.ye import (
    get_hdfs_conn,
    get_spark,
    hive_execute,
    hive_to_pandas,
    pandas_to_parquet,
    slack_send,
    get_secrets
)

In [3]:
import os

In [4]:
os.environ['http_proxy'] = proxies['http']
os.environ['https_proxy'] = proxies['https']

In [5]:
#!pip install transformers

In [6]:
import torch
from transformers import (
    AdamW,
    AutoModel,
    get_linear_schedule_with_warmup,
    AutoTokenizer,
    AutoConfig
)
import torch.nn.functional as F


# EVENT EMBEDDING

# data

In [8]:
team_ids = (
    "APL00000C9YLLYY6ST1C",
    "APL00000C32HCIJVBYTC",
    "APL00000BPQAQNPGIL8G",
    "APL00000BJHW4MH1CYRK",
    "APL00000BJQHK6DLFXFK",
    "APL00000BJ7880KK6QKG",
    "APL00000BP2XDZPWNLDS",
    "APL00000BJIDIL2TP62O",
    "APL00000BLE4UX86YN7K",
    "APL00000BK0HM6I9TKW0",
    "APL00000BJ7SZYXMPKW0",
    "APL00000CBG69D7ZF0N4",
    "APL00000D2RF7YKOULFK",
    "APL00000BWL870Y1IDXC",
    "APL00000BJQDP1MB591C",
    "APL00000BJWB7QDVTKHS",
    "APL00000C9W84KVCT6V4",
    "APL00000BJAN9OTD7W8W",
    "APL00000CQFNC0VHSWE8",
    "APL00000BJIFB8GKCVSW",
    "APL00000D7VQ7G7XIW3K"
)

In [28]:
def get_tables(table_nm:str):
    query = f"""
        SELECT  CONCAT(luna_id, "_", "tmap") as luna_id_key,
                luna_id,
                tmap_profile as profile_templates,
                'tmap' as source_domain
        FROM x1113099.user_retrieval_profile_tmap_text
        WHERE luna_id in {team_ids}
        
        UNION ALL
        
        SELECT  CONCAT(luna_id, "_", "adot") as luna_id_key,
                luna_id, 
                adot_profile as profile_templates,
                'adot' as source_domain
        FROM x1113099.user_retrieval_profile_adot_text
        WHERE luna_id in {team_ids}
        
        UNION ALL
        
        SELECT  CONCAT(luna_id, "_", "xdr") as luna_id_key,
                luna_id, 
                xdr_profile as profile_templates,
                'xdr' as source_domain
        FROM x1113099.user_retrieval_profile_xdr_text
        WHERE luna_id in {team_ids}
        
        UNION ALL
        
        SELECT  CONCAT(luna_id, "_", "tdeal") as luna_id_key,
                luna_id,
                tdeal_profile as profile_templates,
                'tdeal' as source_domain
        FROM adot_reco_dev.adotUser_tdeal_profile_temp
        WHERE luna_id in {team_ids}
    """
    return bq_to_pandas(query)

In [29]:
data_df = get_tables(table_nm="adotUser_tdeal_profile_temp")

query: 
        SELECT  CONCAT(luna_id, "_", "tmap") as luna_id_key,
                luna_id,
                tmap_profile as profile_templates,
                'tmap' as source_domain
        FROM x1113099.user_retrieval_profile_tmap_text
        WHERE luna_id in ('APL00000C9YLLYY6ST1C', 'APL00000C32HCIJVBYTC', 'APL00000BPQAQNPGIL8G', 'APL00000BJHW4MH1CYRK', 'APL00000BJQHK6DLFXFK', 'APL00000BJ7880KK6QKG', 'APL00000BP2XDZPWNLDS', 'APL00000BJIDIL2TP62O', 'APL00000BLE4UX86YN7K', 'APL00000BK0HM6I9TKW0', 'APL00000BJ7SZYXMPKW0', 'APL00000CBG69D7ZF0N4', 'APL00000D2RF7YKOULFK', 'APL00000BWL870Y1IDXC', 'APL00000BJQDP1MB591C', 'APL00000BJWB7QDVTKHS', 'APL00000C9W84KVCT6V4', 'APL00000BJAN9OTD7W8W', 'APL00000CQFNC0VHSWE8', 'APL00000BJIFB8GKCVSW', 'APL00000D7VQ7G7XIW3K')
        
        UNION ALL
        
        SELECT  CONCAT(luna_id, "_", "adot") as luna_id_key,
                luna_id, 
                adot_profile as profile_templates,
                'adot' as source_domain
        FROM x11

In [31]:
data_df

,luna_id_key,luna_id,profile_templates,source_domain
0,APL00000BJIFB8GKCVSW_xdr,APL00000BJIFB8GKCVSW,"저는 주로 페이앱,공유 협업,라이브 방송 플랫폼 카테고리와 관련된 앱을 사용했어요",xdr
1,APL00000BJAN9OTD7W8W_xdr,APL00000BJAN9OTD7W8W,"저는 주로 메일,온라인 쇼핑몰,구직,공유 협업,유아/아기,헬스/비대면진료,블로그,기...",xdr
2,APL00000C9W84KVCT6V4_xdr,APL00000C9W84KVCT6V4,"저는 주로 포탈앱,소셜미디어,맛집/음식점, 배달,스타일/패션/뷰티,라이프스타일,공유...",xdr
3,APL00000BJQHK6DLFXFK_xdr,APL00000BJQHK6DLFXFK,"저는 주로 메일,구직,라이프스타일,공유 협업,지도/네비,AI서비스 카테고리와 관련된...",xdr
4,APL00000D7VQ7G7XIW3K_xdr,APL00000D7VQ7G7XIW3K,"저는 주로 영상 OTT,소셜미디어,맛집/음식점, 배달,메일,공유 협업,지도/네비,헬...",xdr
5,APL00000C9YLLYY6ST1C_xdr,APL00000C9YLLYY6ST1C,"저는 주로 소셜미디어,부동산,메일,주식투자,자산 관리/투자,공유 협업,지도/네비,기...",xdr
6,APL00000CBG69D7ZF0N4_xdr,APL00000CBG69D7ZF0N4,"저는 주로 음악,암호화폐,스타일/패션/뷰티,페이앱,주식투자,공유 협업,은행,온라인 ...",xdr
7,APL00000BJIDIL2TP62O_xdr,APL00000BJIDIL2TP62O,"저는 주로 소셜미디어,온라인 서점/e 북,공유 협업,finance_etc,지도/네비...",xdr
8,APL00000C32HCIJVBYTC_xdr,APL00000C32HCIJVBYTC,"저는 주로 communication_etc,game_etc,쿠폰/포인트앱,메일,온라...",xdr
9,APL00000BPQAQNPGIL8G_xdr,APL00000BPQAQNPGIL8G,"저는 주로 소셜미디어,운송(공유차/비행기/버스),맛집/음식점, 배달,메일,공유 협업...",xdr


In [32]:
profile_dict = data_df.set_index("luna_id_key").to_dict()['profile_templates']

In [33]:
profile_dict = data_df.set_index("luna_id").to_dict()['tdeal_profile']

In [34]:
profiles = []
luna_ids = []
for luna_id, profile in profile_dict.items():
    profiles.append(profile)
    luna_ids.append(luna_id)

# Inference

In [35]:
model = AutoModel.from_pretrained('BM-K/KoDiffCSE-RoBERTa')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoDiffCSE-RoBERTa')

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [38]:
from tqdm.notebook import tqdm

In [39]:
def batch_embedd(profiles:list, luna_ids:list, batch_size = 32, method="cls"):
    embedding_result = []
    luna_id_result = []
    with torch.no_grad():
        for i in range(0, len(profiles), batch_size):
            batch = profiles[i:i+batch_size]
            batch_luna_ids = luna_ids[i:i+batch_size]
            inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt").to(device)
            outputs = model(**inputs, return_dict=True)
            #embeddings, _ = model(**inputs, return_dict=False)
            if method =='cls':
                embeddings = outputs.last_hidden_state[:, 0, :]  
                embeddings = F.normalize(embeddings, p=2, dim=1)
                embeddings = embeddings.unsqueeze(1)
            elif method =='mean_pool':
                embeddings = torch.mean(outputs.last_hidden_state, dim=1)
                embeddings = F.normalize(embeddings, p=2, dim=1)
                embeddings = embeddings.unsqueeze(1) # Shape: (batch_size, 768)
            embedding_result.extend(embeddings.cpu().numpy())
            luna_id_result.extend(batch_luna_ids)
            
    return embedding_result, luna_id_result

In [40]:
embedding_result, luna_id_result = batch_embedd(profiles, luna_ids)

In [42]:
# id단위로 reformulate

In [45]:
luna_id_result[0]

'APL00000BJIFB8GKCVSW_xdr'

In [46]:
result = []
for i in range(len(luna_id_result)):
    result.append({"profile_text": profiles[i], "profile_vector": embedding_result[i].squeeze(0), "luna_id":luna_id_result[i].split('_')[0], 'source_domain':luna_id_result[i].split('_')[1], 'dt':'temp'})

In [49]:
import pandas as pd
df = pd.DataFrame(result)

In [50]:
PROJECT_ID = "skt-datahub"
db_name = "adot_reco_dev"
table_name = "team_profiles_table"

In [51]:
pandas_to_bq(pd_df = df, destination=f"{PROJECT_ID}.{db_name}.{table_name}")

# nudege_api

In [52]:
import requests
import pandas as pd
import json

In [ ]:
event_id= pd.DataFrame(pd.DataFrame(json.loads(response.text)['data'])['row'].tolist()).fillna('')
event_id.columns=['product','scenario_id','nudge_id','event_id','event_id_description','variable','nudge_title','nudge_description']
event_id=event_id[event_id['event_id_description']!=''].reset_index().drop("index",axis=1)

input_ids=event_id.event_id.tolist()
input_=event_id.event_id_description.tolist()

In [62]:
response = requests.post(url, headers=headers, data="{}")

In [65]:
api_text = json.loads(response.text)

In [76]:
from typing import List, Dict
def get_api_cols(infos:List[Dict]):
    result = list()
    for info in infos:
        result.append(info['name'])
    return result

In [85]:
api_info = api_text['info']
api_data = api_text['data']
col_names = get_api_cols(api_info)

In [86]:
col_names

['product',
 'scenario_id',
 'nudge_id',
 'event_id',
 'event_id_description',
 'variable',
 'mandatory_yn',
 'nudge_title',
 'nudge_description']

In [93]:
api_final_result = []
for api_datum in api_data:
    temp_dict = dict()
    data = api_datum['row']
    for col_name, datum in zip(col_names, data):
        if col_name == 'event_id_description' and datum is None:
            temp_dict = None
            break
        else:
            temp_dict[col_name] = datum
    if temp_dict is not None:
        api_final_result.append(temp_dict)
        

In [96]:
api_data = pd.DataFrame(api_final_result)

In [88]:
api_data[0]['row']

{'row': ['agentOffering',
  'general',
  'nudge_5174',
  'EVT_15701',
  None,
  '{"titleTextPolite":"{{일반선톡_092}}","utteranceGuideApi":"{{UT_STATIC}}"}',
  'N',
  '일반선톡_091~092',
  '#일반선톡 #일요일 #오후']}

In [81]:
pd.DataFrame(api_text['data']['row'], columns=col_names)

TypeError: list indices must be integers or slices, not str

In [73]:
event_id= pd.DataFrame(pd.DataFrame(json.loads(response.text)['data'])['row'].tolist()).fillna('')


['nudge_description',
 'nudge_id',
 'product',
 'nudge_title',
 'variable',
 'mandatory_yn',
 'event_id',
 'scenario_id',
 'event_id_description']